## ISLP Chapter 5 - Resampling

### Applied exercise 6

In [1]:
# import relevant statistical packages
import numpy as np
import pandas as pd

In [2]:
# import relevant data visualisation packages
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
# load Default dataset
# url = "/Users/arpanganguli/Documents/Professional/Finance/ISLR/Datasets/Default.csv"
url = "../../Data/Default.csv"
Default = pd.read_csv(url, index_col = 'Unnamed: 0')

In [4]:
Default.head()

,default,student,balance,income
1,No,No,729.526495,44361.625074
2,No,Yes,817.180407,12106.134700
3,No,No,1073.549164,31767.138947
4,No,No,529.250605,35704.493935
5,No,No,785.655883,38463.495879


In [5]:
Default.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 390.6+ KB


In [6]:
dfX = Default[['student', 'balance','income']]
dfX = pd.get_dummies(data = dfX, drop_first=True)
dfy = Default['default']

In [7]:
dfX.head()

,balance,income,student_Yes
1,729.526495,44361.625074,0
2,817.180407,12106.134700,1
3,1073.549164,31767.138947,0
4,529.250605,35704.493935,0
5,785.655883,38463.495879,0


In [8]:
dfy.head()

1    No
2    No
3    No
4    No
5    No
Name: default, dtype: object

### 6.a. Estimated standard errors for coefficients associated with income and balance in a mulitple logit model

In [9]:
import statsmodels.api as sm

In [10]:
X = dfX[['balance', 'income']]
X = sm.add_constant(X)
y = pd.get_dummies(dfy, drop_first=True)

In [11]:
glmfit = sm.GLM(y, X, family=sm.families.Binomial()).fit()

In [12]:
glmfit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    Yes   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Sun, 15 Oct 2023   Deviance:                       1579.0
Time:                        21:23:18   Pearson chi2:                 6.95e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1256
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0.000     24.835      0.000       0.005       0.006
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
==============================================================================
"""

In [13]:
estimated_std_err = np.array(glmfit.params / glmfit.tvalues)

In [14]:
estimated_std_err

array([4.34772469e-01, 2.27381384e-04, 4.98524546e-06])

### 6.b. Writing function bootfn() which inputs Default data and index of the observations and returns estimates of the standard errors of the regression coefficients

In [15]:
def bootfn(data, index):
    X = data[['balance', 'income']]
    X = sm.add_constant(X)
    y = pd.get_dummies(data['default'], drop_first=True)
    X_train = X.iloc[index]
    y_train = y.iloc[index]
    glmfit = sm.GLM(y_train, X_train, family=sm.families.Binomial()).fit()
    estimated_std_err = np.array(glmfit.params / glmfit.tvalues)
    return estimated_std_err

In [16]:
bootfn(Default, list(range(1,10000)))

array([4.34775145e-01, 2.27384181e-04, 4.98521904e-06])

### 6.c. Bootstrapping the bootfn() function to estimate the standard errors of the regression coefficients

In [17]:
from sklearn.utils import resample

In [18]:
std_err_df = pd.DataFrame()
idx = list(range(10000))

In [19]:
for i in range(1000):
    std_temp = bootfn(Default, resample(idx, replace=True))
    std_err_df = std_err_df.append([std_temp])

/var/folders/_c/g4nn9vsd285_gcjw3nh8c8kr0000gn/T/ipykernel_60232/910898906.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_err_df = std_err_df.append([std_temp])
/var/folders/_c/g4nn9vsd285_gcjw3nh8c8kr0000gn/T/ipykernel_60232/910898906.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_err_df = std_err_df.append([std_temp])
/var/folders/_c/g4nn9vsd285_gcjw3nh8c8kr0000gn/T/ipykernel_60232/910898906.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_err_df = std_err_df.append([std_temp])
/var/folders/_c/g4nn9vsd285_gcjw3nh8c8kr0000gn/T/ipykernel_60232/910898906.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_err

In [20]:
std_err_df.reset_index(drop=True, inplace=True)
std_err_df.columns = ['intercept', 'balance', 'income']

In [21]:
std_err_df.head()

,intercept,balance,income
0,0.426060,0.000218,0.000005
1,0.443096,0.000230,0.000005
2,0.425023,0.000222,0.000005
3,0.442896,0.000232,0.000005
4,0.470979,0.000240,0.000005


In [22]:
std_err_df.shape

(1000, 3)

In [23]:
std_err_df.mean()

intercept    0.437411
balance      0.000229
income       0.000005
dtype: float64

### 6.d. As we can see, the bootstrap of the standard error estimates are close to standard errors of logistic regression coefficients.